<a href="https://colab.research.google.com/github/TejoramV/CS6910_Assignment2/blob/main/Part-A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, BatchNormalization, Dense, Flatten

!pip install wandb
import wandb
from wandb.keras import WandbCallback
from PIL import Image

     |████████████████████████████████| 1.8 MB 4.7 MB/s 
     |████████████████████████████████| 181 kB 34.0 MB/s 
     |████████████████████████████████| 144 kB 45.8 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=6d733907f2c0f3f65151fb557f3d7ddc9084fc121937acc132c9c7c72baac5d8
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def buildModel(filter_size_needed=11, filter_scale="half", num_filters=32, filter_multiplier=1, activation_func="relu", dropout=0.2, batch_norm=False, dense_size=64, num_classes=10, image_size=200):
    model = Sequential()
    if filter_scale=="half":
      scale=-2
    elif filter_scale=="double":
      scale=2
    else:
      scale=0
    
    for i in range(5):
        filter_dim = filter_size_needed + (scale)*i
        filter_size = (filter_dim, filter_dim)
        if i==0:
            model.add(Conv2D(num_filters, filter_size, input_shape=(image_size, image_size, 3), data_format="channels_last"))
        else:
            model.add(Conv2D(num_filters, filter_size))
        if batch_norm:
            model.add(BatchNormalization())
        model.add(Activation(activation_func))
        model.add(MaxPooling2D(pool_size=(2,2)))
        num_filters = int(num_filters * filter_multiplier)
    
    model.add(Flatten())
    model.add(Dense(dense_size))
    model.add(Dropout(dropout))
    model.add(Activation(activation_func))
    model.add(Dense(num_classes))
    model.add(Activation("softmax"))

    return model

In [4]:
def load_dataset(augment_data=False):
    train_dir = "/content/drive/MyDrive/inaturalist_12K/train"
    test_dir = "/content/drive/MyDrive/inaturalist_12K/val"

    if augment_data:
        train = ImageDataGenerator(rescale=1./255, rotation_range=20, zoom_range=0.2, shear_range=0.2, validation_split=0.1,horizontal_flip=True)
        test = ImageDataGenerator(rescale=1./255)

    else:
        train = ImageDataGenerator(rescale=1./255, validation_split=0.1)
        test = ImageDataGenerator(rescale=1./255)

    train_data = train.flow_from_directory(train_dir, target_size=(200, 200), batch_size=256, subset="training")
    val_data = train.flow_from_directory(train_dir, target_size=(200, 200), batch_size=256, subset="validation")
    test_data = test.flow_from_directory(test_dir, target_size=(200, 200), batch_size=256)
    
    return train_data, val_data, test_data;

In [6]:
def train():

    config_defaults = {
        "filter_size_needed": 11,
        "filter_scale": "half",
        "num_filters": 32,
        "filter_multiplier": 2,
        "activation_func": "relu",
        "augment_data": False,
        "dropout": 0.3,
        "batch_norm": False,
        "epochs": 10,
        "dense_size": 64,
        "lr": 0.001
    }

    wandb.init(config=config_defaults, magic=True)
    config = wandb.config
    filter_size_needed=config.filter_size_needed
    filter_scale=config.filter_scale    
    num_filters=config.num_filters
    filter_multiplier=config.filter_multiplier
    activation_func=config.activation_func
    augment_data=config.augment_data
    dropout=config.dropout
    batch_norm=config.batch_norm
    epochs=config.epochs
    dense_size=config.dense_size
    lr=config.lr
    wandb.run.name = "fil.size_{}_fil.scale_{}_NO.fil_{}_fil.mul_{}_activ_{}_aug.data_{}_dropout_{}_batch.norm_{}_e_{}_dense_{}_lr_{}".format(filter_size_needed, \
                                                                                                                                              filter_scale, \
                                                                                                                                              num_filters, \
                                                                                                                                              filter_multiplier, \
                                                                                                                                              activation_func, \
                                                                                                                                              augment_data, \
                                                                                                                                              dropout, \
                                                                                                                                              batch_norm, \
                                                                                                                                              epochs, \
                                                                                                                                              dense_size, \
                                                                                                                                              lr)
    train_data, val_data, test_data = load_dataset(augment_data=augment_data)
    model = buildModel(filter_size_needed=filter_size_needed, filter_scale=filter_scale, num_filters=num_filters, filter_multiplier=filter_multiplier,
                      activation_func=activation_func, dropout=dropout, batch_norm=batch_norm, dense_size=dense_size)
    model.compile(optimizer=keras.optimizers.Adam(config.lr), loss="categorical_crossentropy", metrics="categorical_accuracy")
    model.fit(train_data, epochs=epochs, validation_data=val_data, callbacks=[WandbCallback()])

In [7]:
def test():

    config_defaults = {
        "filter_size_needed": 11,
        "filter_scale": "half",
        "num_filters": 32,
        "filter_multiplier": 2,
        "activation_func": "relu",
        "augment_data": False,
        "dropout": 0.3,
        "batch_norm": False,
        "epochs": 10,
        "dense_size": 64,
        "lr": 0.001
    }

    wandb.init(config=config_defaults, magic=True)
    config = wandb.config
    filter_size_needed=config.filter_size_needed
    filter_scale=config.filter_scale    
    num_filters=config.num_filters
    filter_multiplier=config.filter_multiplier
    activation_func=config.activation_func
    augment_data=config.augment_data
    dropout=config.dropout
    batch_norm=config.batch_norm
    epochs=config.epochs
    dense_size=config.dense_size
    lr=config.lr

    wandb.run.name = "fil.size_{}_fil.scale_{}_NO.fil_{}_fil.mul_{}_activ_{}_aug.data_{}_dropout_{}_batch.norm_{}_e_{}_dense_{}_lr_{}".format(filter_size_needed, \
                                                                                                                                              filter_scale, \
                                                                                                                                              num_filters, \
                                                                                                                                              filter_multiplier, \
                                                                                                                                              activation_func, \
                                                                                                                                              augment_data, \
                                                                                                                                              dropout, \
                                                                                                                                              batch_norm, \
                                                                                                                                              epochs, \
                                                                                                                                              dense_size, \
                                                                                                                                              lr)

    train_data, val_data, test_data = load_dataset(augment_data=augment_data)
    model = buildModel(filter_size_needed=filter_size_needed, filter_scale=filter_scale, num_filters=num_filters, filter_multiplier=filter_multiplier,
                      activation_func=activation_func, dropout=dropout, batch_norm=batch_norm, dense_size=dense_size)
    model.compile(optimizer=keras.optimizers.Adam(config.lr), loss="categorical_crossentropy", metrics="categorical_accuracy")
    model.fit(train_data, epochs=epochs, validation_data=val_data, callbacks=[WandbCallback()])

    print("Evaluating Model:")
    model.evaluate(test_data, batch_size=256)

In [ ]:
sweep_config = {
    'metric': {'name': 'val_Accuracy','goal':'maximize'},
    "method": "grid",
    "project": "CS6910_Assignment2-Part-A",
    "parameters": {
        "filter_size_needed": {
            "values": [5,7]
        },
        "filter_scale": {
            "values": ["same"]
        },                
        "num_filters": {
            "values": [32,64]
        },
        "filter_multiplier": {
            "values": [1,2]
        },
        "activation_func": {
            "values": ["relu"]
        },        
        "augment_data": {
            "values": [True]
        },
        "dropout": {
            "values": [0.1,0.3]
        },
        "batch_norm": {
            "values": [True]
        },
        "epochs": {
            "values": [10]
        },
        "dense_size": {
            "values": [64]
        },
        "lr": {
            "values": [0.001,0.0001]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="CS6910_Assignment2-Part-A")

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. 'Val Accuracy' is not of type 'object'


Create sweep with ID: d581e45s
Sweep URL: https://wandb.ai/tejoram/CS6910_Assignment2-roughtest/sweeps/d581e45s


In [ ]:
wandb.agent(sweep_id, function=train)

In [ ]:
#Best params
sweep_config = {
    'metric': {'name': 'val_Accuracy','goal':'maximize'},
    "method": "grid",
    "project": "CS6910_Assignment2-Part-A",
    "parameters": {
        "filter_size_needed": {
            "values": [11]
        },
        "filter_scale": {
            "values": ["same"]
        },                
        "num_filters": {
            "values": [32]
        },
        "filter_multiplier": {
            "values": [2]
        },
        "activation_func": {
            "values": ["relu"]
        },        
        "augment_data": {
            "values": [True]
        },
        "dropout": {
            "values": [0.3]
        },
        "batch_norm": {
            "values": [True]
        },
        "epochs": {
            "values": [10]
        },
        "dense_size": {
            "values": [64]
        },
        "lr": {
            "values": [0.01]
        }
    }
}


sweep_id = wandb.sweep(sweep_config, project="CS6910_Assignment2-Part-A")

In [ ]:
wandb.agent(sweep_id, function=test)